In [16]:
import pandas as pd
import numpy as np
from numpy import random
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize.toktok import ToktokTokenizer

import glob
import os

DATA_FOLDER = '../data/'

In [26]:
# all_docs = glob.iglob(DATA_FOLDER + 'FIPS/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [2]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [15]:
len(all_docs)

1194423

In [25]:
DATA_FOLDER = os.path.join('../data/', 'FIPS/Inventions patents_txt_output/0c/0c/')
all_docs = glob.iglob(DATA_FOLDER + '*.txt', recursive=True)

In [26]:
stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
stop_list.extend(list(string.punctuation))

punkts = [s for s in string.punctuation if s not in '.!?']


class Corpus(gensim.corpora.TextCorpus):
    def get_texts(self):
        for filename in self.input: # for each relevant file
            yield tokenize(open(filename).read())

            
def validate_word(word):
    word = word.replace("«", "").replace("»", "")
    return word
            
            
def tokenize(file_text):
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [validate_word(i) for i in tokens if (i not in stop_list and not i.isdigit())]
    tokens = filter(lambda s: s != '', tokens)
    
    return tokens
    

In [30]:
mycorpus = Corpus(all_docs)

In [29]:
mycorpus.dictionary.token2id

{'концентрат': 0,
 'смазочно-охлаждающей': 1,
 'жидкость': 2,
 'механический': 3,
 'обработка': 4,
 'металл': 5,
 'использование': 6,
 'содержать': 7,
 'мас': 8,
 'минеральный': 9,
 'масло': 10,
 '15-20': 11,
 'продукт': 12,
 'конденсация': 13,
 'ненасыщать': 14,
 'жирный': 15,
 'кислота': 16,
 "2,2'-иминодиэтанола": 17,
 'мольный': 18,
 'соотношение': 19,
 '2,5-5,0': 20,
 '16-28': 21,
 'растительный': 22,
 'фракция': 23,
 'c12-c20': 24,
 '1-5': 25,
 'гликоли': 26,
 '2-6': 27,
 'полиоксиэтилированный': 28,
 'алкилфенол': 29,
 '0,1-4,0': 30,
 'взаимодействие': 31,
 'кислый': 32,
 'эфир': 33,
 'борный': 34,
 'алканоламин': 35,
 '15-26': 36,
 'двузамещать': 37,
 'алифатический': 38,
 'фосфорный': 39,
 '1-7': 40,
 'пеногаситель': 41,
 '0,05-0,3': 42,
 'вода': 43,
 'остальное': 44,
 'технический': 45,
 'результат': 46,
 'улучшение': 47,
 'санитарно-гигиенических': 48,
 'характеристика': 49,
 'повышение': 50,
 'устойчивость': 51,
 'влияние': 52,
 'солея': 53,
 'жесткость': 54,
 'сохранение':

In [33]:
list(all_docs)

[]

In [28]:
wv = gensim.models.KeyedVectors.load_word2vec_format(
    "~/Documents/wordvecs/ruwikiruscorpora_0_300_20.bin",
    binary=True)
wv.init_sims(replace=True)

In [29]:
from itertools import islice
list(islice(wv.vocab, 13000, 13020))

['кампус_NOUN',
 'макдональд_NOUN',
 'девятнадцатый_ADJ',
 'электротехнический_ADJ',
 'проникнутый_ADJ',
 'полушубок_NOUN',
 'клавишный_ADJ',
 'годичный_ADJ',
 'знаменательный_ADJ',
 'горох_NOUN',
 'задрожать_VERB',
 'вавилонский_ADJ',
 'скорпион_NOUN',
 'сворачивать_VERB',
 'недалекий_ADJ',
 'оглядеться_VERB',
 'четырехлетний_ADJ',
 'мамочка_NOUN',
 'котелок_NOUN',
 'news_NOUN']

In [31]:
wv.most_similar(positive=['силос_NOUN'], restrict_vocab=100000, topn=30)[0:20]

[('силосный_ADJ', 0.657220721244812),
 ('корнеплод_NOUN', 0.5938321948051453),
 ('компост_NOUN', 0.5927795767784119),
 ('корм_NOUN', 0.5741930603981018),
 ('удобрение_NOUN', 0.5622274875640869),
 ('ботва_NOUN', 0.5541197061538696),
 ('аммиачный::селитра_NOUN', 0.5507166385650635),
 ('зерно_NOUN', 0.5447162389755249),
 ('бурт_NOUN', 0.5432900190353394),
 ('комбикорм_NOUN', 0.5354993343353271),
 ('жмых_NOUN', 0.5318485498428345),
 ('овощехранилище_NOUN', 0.5261991024017334),
 ('навоз_NOUN', 0.5203781127929688),
 ('отруби_NOUN', 0.5183936953544617),
 ('сушилка_NOUN', 0.5149815678596497),
 ('птичий::помет_NOUN', 0.514065682888031),
 ('клубень_NOUN', 0.5139718651771545),
 ('кукуруза_NOUN', 0.507766604423523),
 ('гранулированный_ADJ', 0.5018250346183777),
 ('брикет_NOUN', 0.4995535910129547)]

### для нового тектса нужно обязательно: токенизировать, разбить на предложения, лемматизировать и разметить по частям речи при помощи Mystem

In [34]:
!./mystem -i --format json test.txt

[{"analysis":[{"lex":"программа","gr":"S,жен,неод=им,ед"}],"text":"Программа"},{"analysis":[],"text":"MyStem"},{"analysis":[{"lex":"производить","gr":"V=непрош,ед,изъяв,3-л,несов,пе"}],"text":"производит"},{"analysis":[{"lex":"морфологический","gr":"A=вин,ед,полн,муж,неод"},{"lex":"морфологический","gr":"A=им,ед,полн,муж"}],"text":"морфологический"},{"analysis":[{"lex":"анализ","gr":"S,муж,неод=вин,ед"},{"lex":"анализ","gr":"S,муж,неод=им,ед"}],"text":"анализ"},{"analysis":[{"lex":"текст","gr":"S,муж,неод=род,ед"}],"text":"текста"},{"analysis":[{"lex":"на","gr":"PR="},{"lex":"на","gr":"PART="}],"text":"на"},{"analysis":[{"lex":"русский","gr":"A=пр,ед,полн,муж"},{"lex":"русский","gr":"A=пр,ед,полн,сред"},{"lex":"русский","gr":"S,муж,од=пр,ед"}],"text":"русском"},{"analysis":[{"lex":"язык","gr":"S,муж,неод=пр,ед"},{"lex":"язык","gr":"S,муж,од=пр,ед"}],"text":"языке"},{"analysis":[{"lex":"она","gr":"SPRO,ед,3-л,жен=им"}],"text":"Она"},{"analysis":[{"lex":"уметь","gr":"V,несов,нп=непрош,ед